In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Custom MSE to handle -999 placeholders
def custom_mse(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, -999), tf.float32)
    error = tf.square(y_pred - y_true) * mask
    return tf.reduce_sum(error) / tf.reduce_sum(mask)

# Load and prepare data
lines = []
with open('Final_static_dataset.txt', 'r') as file:
    lines = file.readlines()

steering_angles, throttles, x_coords_list, flags_list = [], [], [], []

for line in lines:
    if line.strip():
        parts = line.split(': ')
        image_number, data = parts[0], parts[1]
        control_values, points, flags = eval(data)
        steering_angles.append(control_values[0])
        throttles.append(control_values[1])
        x_coords_list.append([point[0] if point else np.nan for point in points][::-1])  # Reverse the order here

df = pd.DataFrame(x_coords_list, columns=['x_farthest', 'x_middle2', 'x_middle1', 'x_closest'])
df['steering_angle'], df['throttle'] = steering_angles, throttles

for col in ['x_farthest', 'x_middle2', 'x_middle1', 'x_closest']:
    df[f'{col}_missing'] = df[col].isna().astype(int)

for col in ['x_farthest', 'x_middle2', 'x_middle1', 'x_closest']:
    df[f'next_{col}'] = df[col].shift(-1)

df = df[:-1]

# Replace missing values with -999
df.fillna(-999, inplace=True)

# Separate features and target variables
X = df.drop(columns=[f'next_{col}' for col in ['x_farthest', 'x_middle2', 'x_middle1', 'x_closest']] +
                   [f'{col}_missing' for col in ['x_farthest', 'x_middle2', 'x_middle1', 'x_closest']])
y_coords = df[[f'next_{col}' for col in ['x_farthest', 'x_middle2', 'x_middle1', 'x_closest']]]

# Split data into train and test sets
X_train, X_test, y_coords_train, y_coords_test = train_test_split(
    X, y_coords, test_size=0.2, random_state=42)

# Replace -999 with NaN for scaling
X_train.replace(-999, np.nan, inplace=True)
X_test.replace(-999, np.nan, inplace=True)
y_coords_train.replace(-999, np.nan, inplace=True)
y_coords_test.replace(-999, np.nan, inplace=True)

# Apply MinMaxScaler
feature_scaler = MinMaxScaler()
coords_scaler = MinMaxScaler()

scaled_features_train = feature_scaler.fit_transform(X_train)
scaled_features_test = feature_scaler.transform(X_test)

scaled_coords_train = coords_scaler.fit_transform(y_coords_train)
scaled_coords_test = coords_scaler.transform(y_coords_test)

# Replace NaN back with -999 after scaling
X_train_scaled = pd.DataFrame(scaled_features_train, columns=X_train.columns).fillna(-999)
X_test_scaled = pd.DataFrame(scaled_features_test, columns=X_test.columns).fillna(-999)

y_coords_train_scaled = pd.DataFrame(scaled_coords_train, columns=y_coords_train.columns).fillna(-999)
y_coords_test_scaled = pd.DataFrame(scaled_coords_test, columns=y_coords_test.columns).fillna(-999)

# Build the model
inputs = Input(shape=(X_train_scaled.shape[1],))
x = Dense(128, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
coords_output = Dense(4, name='coords_output')(x)  # Output for coordinates

model = Model(inputs=inputs, outputs=coords_output)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss=custom_mse,
              metrics=['mean_absolute_error'])

# Train the model
model.fit(X_train_scaled, y_coords_train_scaled, validation_split=0.2, epochs=15, batch_size=32)

# Evaluate the model
results = model.evaluate(X_test_scaled, y_coords_test_scaled)
print(f"Test Loss, Test MAE: {results}")

# Predict coordinates
predictions_coords = model.predict(X_test_scaled)

# Evaluate coordinates
# Replace -999 with NaN before inverse transform
y_coords_test_scaled.replace(-999, np.nan, inplace=True)
predictions_coords_df = pd.DataFrame(predictions_coords, columns=y_coords_test.columns)
predictions_coords_df.replace(-999, np.nan, inplace=True)

# Inverse transform the coordinates
y_coords_test_inv = coords_scaler.inverse_transform(y_coords_test_scaled)
predictions_coords_inv = coords_scaler.inverse_transform(predictions_coords_df)

# Replace NaN back with -999 after inverse transform
y_coords_test_inv = pd.DataFrame(y_coords_test_inv, columns=y_coords_test.columns).fillna(-999)
predictions_coords_inv = pd.DataFrame(predictions_coords_inv, columns=y_coords_test.columns).fillna(-999)

# Apply the mask after inverse transform
mask = np.all(y_coords_test_inv != -999, axis=1)
filtered_y_test = y_coords_test_inv[mask]
filtered_y_pred = predictions_coords_inv[mask]

filtered_mse = mean_squared_error(filtered_y_test, filtered_y_pred)
filtered_rmse = np.sqrt(filtered_mse)
filtered_mae = mean_absolute_error(filtered_y_test, filtered_y_pred)
filtered_r2 = r2_score(filtered_y_test, filtered_y_pred)

print(f"Filtered Mean Squared Error: {filtered_mse}")
print(f"Filtered Root Mean Squared Error: {filtered_rmse}")
print(f"Filtered Mean Absolute Error: {filtered_mae}")
print(f"Filtered R-squared Score: {filtered_r2}")


Epoch 1/15


2024-07-18 23:17:49.707687: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


414/414 [==============================] - 1s 1ms/step - loss: 1.0967 - mean_absolute_error: 52.0202 - val_loss: 0.0498 - val_mean_absolute_error: 50.3277
Epoch 2/15
414/414 [==============================] - 0s 882us/step - loss: 0.1919 - mean_absolute_error: 51.6605 - val_loss: 0.0244 - val_mean_absolute_error: 50.2881
Epoch 3/15
414/414 [==============================] - 0s 870us/step - loss: 0.0604 - mean_absolute_error: 51.5573 - val_loss: 0.0217 - val_mean_absolute_error: 50.2925
Epoch 4/15
414/414 [==============================] - 0s 855us/step - loss: 0.0341 - mean_absolute_error: 51.5305 - val_loss: 0.0201 - val_mean_absolute_error: 50.2859
Epoch 5/15
414/414 [==============================] - 0s 857us/step - loss: 0.0283 - mean_absolute_error: 51.5211 - val_loss: 0.0207 - val_mean_absolute_error: 50.2888
Epoch 6/15
414/414 [==============================] - 0s 852us/step - loss: 0.0266 - mean_absolute_error: 51.5207 - val_loss: 0.0210 - val_mean_absolute_error: 50.2962
Epoch

In [3]:
df

,x_farthest,x_middle2,x_middle1,x_closest,steering_angle,throttle,x_farthest_missing,x_middle2_missing,x_middle1_missing,x_closest_missing,next_x_farthest,next_x_middle2,next_x_middle1,next_x_closest
0,128,128.0,129.0,126.0,0.0,0.500000,0,0,0,0,127.0,127.0,128.0,129.0
1,127,127.0,128.0,129.0,0.0,0.500000,0,0,0,0,127.0,128.0,129.0,129.0
2,127,128.0,129.0,129.0,0.0,0.500000,0,0,0,0,128.0,128.0,129.0,129.0
3,128,128.0,129.0,129.0,0.0,0.500000,0,0,0,0,127.0,128.0,128.0,129.0
4,127,128.0,128.0,129.0,0.0,0.500000,0,0,0,0,127.0,127.0,129.0,129.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20683,81,76.0,74.0,69.0,0.0,-0.455594,0,0,0,0,72.0,70.0,68.0,62.0
20684,72,70.0,68.0,62.0,0.0,0.000000,0,0,0,0,64.0,63.0,64.0,62.0
20685,64,63.0,64.0,62.0,0.0,-0.500000,0,0,0,0,57.0,56.0,60.0,55.0
20686,57,56.0,60.0,55.0,0.0,-0.500000,0,0,0,0,49.0,52.0,57.0,54.0


In [6]:
y_coords_train_scaled

,next_x_farthest,next_x_middle2,next_x_middle1,next_x_closest
0,0.798165,0.785388,0.715596,0.590698
1,0.527523,0.575342,0.688073,0.786047
2,0.137615,0.219178,0.376147,-999.000000
3,0.018349,0.059361,0.133028,0.293023
4,0.105505,0.219178,0.394495,-999.000000
...,...,...,...,...
16545,0.385321,0.401826,0.275229,0.255814
16546,0.646789,0.561644,0.490826,0.348837
16547,0.821101,0.680365,0.541284,0.441860
16548,0.724771,0.684932,0.807339,-999.000000


### without normalization

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def custom_mse(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, -999), tf.float32) 
    error = tf.square(y_pred - y_true) * mask  
    return tf.reduce_sum(error) / tf.reduce_sum(mask)  


lines = []
with open('Final_static_dataset.txt', 'r') as file:
    lines = file.readlines()

# Initialize lists to hold the data
steering_angles = []
throttles = []
x_coords_list = [] 
flags_list = []

# Process each line in the file
for line in lines:
    if line.strip():  # Check if the line is not empty
        parts = line.split(': ')
        image_number, data = parts[0], parts[1]
        control_values, points, flags = eval(data)
        
        # Append only the first two control values to the respective lists
        steering_angles.append(control_values[0])
        throttles.append(control_values[1])
        
        # Extract x-coordinates and flags from points
        x_coords_list.append([point[0] for point in points])
        flags_list.append(flags)

df = pd.DataFrame(x_coords_list, columns=['x_closest', 'x_middle1', 'x_middle2', 'x_farthest'])
df['steering_angle'], df['throttle'] = steering_angles, throttles

for col in ['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']:
    df[f'next_{col}'] = df[col].shift(-1)
df = df[:-1] 
df.fillna(-999, inplace=True)


In [2]:
df

,x_closest,x_middle1,x_middle2,x_farthest,steering_angle,throttle,next_x_closest,next_x_middle1,next_x_middle2,next_x_farthest
0,126,129.0,128.0,128.0,0.0,0.500000,129.0,128.0,127.0,127.0
1,129,128.0,127.0,127.0,0.0,0.500000,129.0,129.0,128.0,127.0
2,129,129.0,128.0,127.0,0.0,0.500000,129.0,129.0,128.0,128.0
3,129,129.0,128.0,128.0,0.0,0.500000,129.0,128.0,128.0,127.0
4,129,128.0,128.0,127.0,0.0,0.500000,129.0,129.0,127.0,127.0
...,...,...,...,...,...,...,...,...,...,...
20683,69,74.0,76.0,81.0,0.0,-0.455594,62.0,68.0,70.0,72.0
20684,62,68.0,70.0,72.0,0.0,0.000000,62.0,64.0,63.0,64.0
20685,62,64.0,63.0,64.0,0.0,-0.500000,55.0,60.0,56.0,57.0
20686,55,60.0,56.0,57.0,0.0,-0.500000,54.0,57.0,52.0,49.0


In [4]:
y_coords = df[[f'next_{col}' for col in ['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']]]
X = df.drop(columns=[f'next_{col}' for col in ['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']])
X_train, X_test, y_coords_train, y_coords_test = train_test_split(
    X, y_coords ,test_size=0.2, random_state=42)


inputs = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
coords_output = Dense(4, name='coords_output')(x) 



model = Model(inputs=inputs, outputs=coords_output)


model.compile(optimizer=Adam(),
              loss={'coords_output': custom_mse},
              metrics={'coords_output': ['mean_absolute_error']})


model.fit(X_train, [y_coords_train], validation_split=0.2, epochs=15, batch_size=32)


results = model.evaluate(X_test, [y_coords_test])

predictions_coords = model.predict(X_test)

Epoch 1/15


2024-07-22 08:47:46.966520: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


414/414 [==============================] - 1s 829us/step - loss: 719.5031 - mean_absolute_error: 69.6012 - val_loss: 121.5504 - val_mean_absolute_error: 61.6729
Epoch 2/15
414/414 [==============================] - 0s 602us/step - loss: 106.2843 - mean_absolute_error: 62.3979 - val_loss: 96.0251 - val_mean_absolute_error: 60.5811
Epoch 3/15
414/414 [==============================] - 0s 578us/step - loss: 91.1785 - mean_absolute_error: 62.1171 - val_loss: 79.3863 - val_mean_absolute_error: 60.0557
Epoch 4/15
414/414 [==============================] - 0s 568us/step - loss: 85.3900 - mean_absolute_error: 62.3713 - val_loss: 81.4200 - val_mean_absolute_error: 60.4738
Epoch 5/15
414/414 [==============================] - 0s 563us/step - loss: 84.1049 - mean_absolute_error: 61.8207 - val_loss: 76.4796 - val_mean_absolute_error: 59.7102
Epoch 6/15
414/414 [==============================] - 0s 565us/step - loss: 82.9782 - mean_absolute_error: 62.6903 - val_loss: 80.7269 - val_mean_absolute_err

In [5]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
y_pred = model.predict(X_test)
coords_pred = y_pred

mask = np.all(y_coords_test != -999, axis=1)
filtered_y_test = y_coords_test[mask]
filtered_y_pred = coords_pred[mask]

filtered_mse = mean_squared_error(filtered_y_test, filtered_y_pred)
filtered_rmse = np.sqrt(filtered_mse)
filtered_mae = mean_absolute_error(filtered_y_test, filtered_y_pred)
filtered_r2 = r2_score(filtered_y_test, filtered_y_pred)

print(f"Filtered Mean Squared Error: {filtered_mse}")
print(f"Filtered Root Mean Squared Error: {filtered_rmse}")
print(f"Filtered Mean Absolute Error: {filtered_mae}")
print(f"Filtered R-squared Score: {filtered_r2}")


130/130 [==============================] - 0s 365us/step
Filtered Mean Squared Error: 66.10278436679296
Filtered Root Mean Squared Error: 8.13036188412256
Filtered Mean Absolute Error: 4.553106772406539
Filtered R-squared Score: 0.9738299085043371


In [8]:
def custom_sequential_predictions(model, X, start_index, n_predictions):
    predictions = []
    current_features = X.iloc[start_index][['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']].values  # Corrected to use directly from DataFrame

    for i in range(n_predictions):
        if start_index + i >= len(X):
            break
        
        
        current_controls = X.iloc[start_index + i][['steering_angle', 'throttle']].values
        features_with_controls = np.concatenate((current_features, current_controls))  
        
       
        current_features_df = pd.DataFrame([features_with_controls], columns=X.columns)
        
    
        predicted_coords = model.predict(current_features_df)[0]
        predictions.append(predicted_coords)
        
        
        current_features = predicted_coords  

    return predictions



custom_df = pd.DataFrame({
    'x_closest': [120] * 10,  
    'x_middle1': [120] * 10,
    'x_middle2': [120] * 10,
    'x_farthest': [120] * 10,
    'steering_angle':[0] * 10,  
    'throttle': np.full(10, 0.5)                
})

# Call the function with custom data
start_index = 0
n_predictions = 10
custom_predictions = custom_sequential_predictions(model, custom_df, start_index, n_predictions)
print("Custom Predictions:", custom_predictions)


1/1 [==============================] - 0s 9ms/step
Custom Predictions: [array([117.677795, 116.920395, 118.86582 , 119.405235], dtype=float32), array([115.33244, 114.04521, 117.28825, 118.62254], dtype=float32), array([113.0606  , 111.28056 , 115.348366, 117.524826], dtype=float32), array([110.91671, 108.58612, 113.1189 , 116.06557], dtype=float32), array([108.92874 , 105.9514  , 110.66641 , 114.246254], dtype=float32), array([107.10786 , 103.381516, 108.052216, 112.0968  ], dtype=float32), array([105.45427 , 100.88878 , 105.33249 , 109.662964], dtype=float32), array([103.96112,  98.48776, 102.55827, 106.99848], dtype=float32), array([102.616844,  96.19257 ,  99.77526 , 104.16017 ], dtype=float32), array([101.406944,  94.01519 ,  97.02356 , 101.20437 ], dtype=float32)]


In [9]:
for i in custom_predictions:
    print(i)

[117.677795 116.920395 118.86582  119.405235]
[115.33244 114.04521 117.28825 118.62254]
[113.0606   111.28056  115.348366 117.524826]
[110.91671 108.58612 113.1189  116.06557]
[108.92874  105.9514   110.66641  114.246254]
[107.10786  103.381516 108.052216 112.0968  ]
[105.45427  100.88878  105.33249  109.662964]
[103.96112  98.48776 102.55827 106.99848]
[102.616844  96.19257   99.77526  104.16017 ]
[101.406944  94.01519   97.02356  101.20437 ]


## without throttle


## 

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def custom_mse(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, -999), tf.float32) 
    error = tf.square(y_pred - y_true) * mask  
    return tf.reduce_sum(error) / tf.reduce_sum(mask)  


lines = []
with open('Final_static_dataset.txt', 'r') as file:
    lines = file.readlines()

# Initialize lists to hold the data
steering_angles = []
throttles = []
x_coords_list = [] 
flags_list = []

# Process each line in the file
for line in lines:
    if line.strip():  # Check if the line is not empty
        parts = line.split(': ')
        image_number, data = parts[0], parts[1]
        control_values, points, flags = eval(data)
        
        # Append only the first two control values to the respective lists
        steering_angles.append(control_values[0])
        throttles.append(control_values[1])
        
        # Extract x-coordinates and flags from points
        x_coords_list.append([point[0] for point in points])
        flags_list.append(flags)

df = pd.DataFrame(x_coords_list, columns=['x_closest', 'x_middle1', 'x_middle2', 'x_farthest'])
df['steering_angle'], df['throttle'] = steering_angles, throttles

for col in ['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']:
    df[f'next_{col}'] = df[col].shift(-1)
df = df[:-1] 
df.fillna(-999, inplace=True)


In [11]:
df

,x_closest,x_middle1,x_middle2,x_farthest,steering_angle,throttle,next_x_closest,next_x_middle1,next_x_middle2,next_x_farthest
0,126,129.0,128.0,128.0,0.0,0.500000,129.0,128.0,127.0,127.0
1,129,128.0,127.0,127.0,0.0,0.500000,129.0,129.0,128.0,127.0
2,129,129.0,128.0,127.0,0.0,0.500000,129.0,129.0,128.0,128.0
3,129,129.0,128.0,128.0,0.0,0.500000,129.0,128.0,128.0,127.0
4,129,128.0,128.0,127.0,0.0,0.500000,129.0,129.0,127.0,127.0
...,...,...,...,...,...,...,...,...,...,...
20683,69,74.0,76.0,81.0,0.0,-0.455594,62.0,68.0,70.0,72.0
20684,62,68.0,70.0,72.0,0.0,0.000000,62.0,64.0,63.0,64.0
20685,62,64.0,63.0,64.0,0.0,-0.500000,55.0,60.0,56.0,57.0
20686,55,60.0,56.0,57.0,0.0,-0.500000,54.0,57.0,52.0,49.0


In [12]:
y_coords = df[[f'next_{col}' for col in ['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']]]

# Exclude 'throttle' from the feature set
X = df.drop(columns=[f'next_{col}' for col in ['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']] + ['throttle'])

X_train, X_test, y_coords_train, y_coords_test = train_test_split(
    X, y_coords ,test_size=0.2, random_state=42)


inputs = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
coords_output = Dense(4, name='coords_output')(x) 



model = Model(inputs=inputs, outputs=coords_output)


model.compile(optimizer=Adam(),
              loss={'coords_output': custom_mse},
              metrics={'coords_output': ['mean_absolute_error']})


model.fit(X_train, [y_coords_train], validation_split=0.2, epochs=15, batch_size=32)


results = model.evaluate(X_test, [y_coords_test])

predictions_coords = model.predict(X_test)

Epoch 1/15
414/414 [==============================] - 0s 761us/step - loss: 985.7506 - mean_absolute_error: 70.0916 - val_loss: 106.2807 - val_mean_absolute_error: 60.7676
Epoch 2/15
414/414 [==============================] - 0s 565us/step - loss: 99.8474 - mean_absolute_error: 62.0323 - val_loss: 85.2376 - val_mean_absolute_error: 60.3760
Epoch 3/15
414/414 [==============================] - 0s 756us/step - loss: 88.4381 - mean_absolute_error: 62.2012 - val_loss: 88.3836 - val_mean_absolute_error: 59.8390
Epoch 4/15
414/414 [==============================] - 0s 655us/step - loss: 83.8533 - mean_absolute_error: 62.1018 - val_loss: 80.9152 - val_mean_absolute_error: 60.8026
Epoch 5/15
414/414 [==============================] - 0s 574us/step - loss: 84.2025 - mean_absolute_error: 62.3609 - val_loss: 97.5067 - val_mean_absolute_error: 61.5916
Epoch 6/15
414/414 [==============================] - 0s 578us/step - loss: 84.8744 - mean_absolute_error: 62.6122 - val_loss: 83.3785 - val_mean_ab

In [13]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
y_pred = model.predict(X_test)
coords_pred = y_pred

mask = np.all(y_coords_test != -999, axis=1)
filtered_y_test = y_coords_test[mask]
filtered_y_pred = coords_pred[mask]

filtered_mse = mean_squared_error(filtered_y_test, filtered_y_pred)
filtered_rmse = np.sqrt(filtered_mse)
filtered_mae = mean_absolute_error(filtered_y_test, filtered_y_pred)
filtered_r2 = r2_score(filtered_y_test, filtered_y_pred)

print(f"Filtered Mean Squared Error: {filtered_mse}")
print(f"Filtered Root Mean Squared Error: {filtered_rmse}")
print(f"Filtered Mean Absolute Error: {filtered_mae}")
print(f"Filtered R-squared Score: {filtered_r2}")


130/130 [==============================] - 0s 316us/step
Filtered Mean Squared Error: 66.86386831747993
Filtered Root Mean Squared Error: 8.177032977644147
Filtered Mean Absolute Error: 4.507114190064607
Filtered R-squared Score: 0.9736754522193171


In [18]:
def custom_sequential_predictions(model, X, start_index, n_predictions):
    predictions = []
    current_features = X.iloc[start_index][['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']].values  # Corrected to use directly from DataFrame

    for i in range(n_predictions):
        if start_index + i >= len(X):
            break
        
        
        current_controls = X.iloc[start_index + i][['steering_angle']].values
        features_with_controls = np.concatenate((current_features, current_controls))  
       
        current_features_df = pd.DataFrame([features_with_controls], columns=X.columns)
        
    
        predicted_coords = model.predict(current_features_df)[0]
        predictions.append(predicted_coords)
        
        
        current_features = predicted_coords  

    return predictions



custom_df = pd.DataFrame({
    'x_closest': [120] * 10,  
    'x_middle1': [120] * 10,
    'x_middle2': [120] * 10,
    'x_farthest': [120] * 10,
    'steering_angle':[0] * 10,                
})

# Call the function with custom data
start_index = 0
n_predictions = 10
custom_predictions = custom_sequential_predictions(model, custom_df, start_index, n_predictions)
print("Custom Predictions:", custom_predictions)


1/1 [==============================] - 0s 9ms/step
Custom Predictions: [array([119.2578  , 120.049835, 119.24323 , 118.78214 ], dtype=float32), array([118.503586, 120.02544 , 118.66386 , 117.74681 ], dtype=float32), array([117.71533, 119.93729, 118.23126, 116.90391], dtype=float32), array([116.878265, 119.78879 , 117.91637 , 116.24704 ], dtype=float32), array([115.98268 , 119.579185, 117.69197 , 115.75995 ], dtype=float32), array([115.02262, 119.30557, 117.53279, 115.42093], dtype=float32), array([113.994835, 118.96399 , 117.41567 , 115.20542 ], dtype=float32), array([112.89822 , 118.550316, 117.3195  , 115.08803 ], dtype=float32), array([111.7333  , 118.06084 , 117.22521 , 115.043564], dtype=float32), array([110.50186, 117.49237, 117.11581, 115.04782], dtype=float32)]


In [19]:
for i in custom_predictions:
    print(i)

[119.2578   120.049835 119.24323  118.78214 ]
[118.503586 120.02544  118.66386  117.74681 ]
[117.71533 119.93729 118.23126 116.90391]
[116.878265 119.78879  117.91637  116.24704 ]
[115.98268  119.579185 117.69197  115.75995 ]
[115.02262 119.30557 117.53279 115.42093]
[113.994835 118.96399  117.41567  115.20542 ]
[112.89822  118.550316 117.3195   115.08803 ]
[111.7333   118.06084  117.22521  115.043564]
[110.50186 117.49237 117.11581 115.04782]


In [43]:
from tensorflow.keras.models import load_model

# Assuming 'model' is your trained model
model.save('NN_NoScale.h5')  # saves to HDF5 file
print("Model saved successfully.")


Model saved successfully.


## NN with normalization and without throttle( Min max Scalar)


In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def custom_mse(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, -999), tf.float32)
    error = tf.square(y_pred - y_true) * mask
    return tf.reduce_sum(error) / tf.reduce_sum(mask)

# Load and preprocess data
lines = []
with open('Final_static_dataset.txt', 'r') as file:
    lines = file.readlines()

steering_angles, throttles, x_coords_list, flags_list = [], [], [], []
for line in lines:
    if line.strip():
        parts = line.split(': ')
        image_number, data = parts[0], parts[1]
        control_values, points, flags = eval(data)
        steering_angles.append(control_values[0])
        throttles.append(control_values[1])
        x_coords_list.append([point[0] for point in points])

df = pd.DataFrame(x_coords_list, columns=['x_closest', 'x_middle1', 'x_middle2', 'x_farthest'])
df['steering_angle'] = steering_angles

for col in ['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']:
    df[f'next_{col}'] = df[col].shift(-1)
df = df[:-1]

# Replace -999 with NaN for scaling purposes
df.replace(-999, np.nan, inplace=True)

y_coords = df[[f'next_{col}' for col in ['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']].copy()]
X = df[['x_closest', 'x_middle1', 'x_middle2', 'x_farthest', 'steering_angle']].copy()

# Split data
X_train, X_test, y_coords_train, y_coords_test = train_test_split(X, y_coords, test_size=0.2, random_state=42)

# Initialize two scalers
#scaler =  MinMaxScaler()

scaler_coords = MinMaxScaler()
scaler_steering = MinMaxScaler()

# Fit the scalers on the appropriate data
scaler_coords.fit(X_train[['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']].fillna(X_train.mean()))
scaler_steering.fit(X_train[['steering_angle']])

# Transform the data
X_train_coords_scaled = scaler_coords.transform(X_train[['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']].fillna(X_train.mean()))
X_train_steering_scaled = scaler_steering.transform(X_train[['steering_angle']])

X_test_coords_scaled = scaler_coords.transform(X_test[['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']].fillna(X_train.mean()))
X_test_steering_scaled = scaler_steering.transform(X_test[['steering_angle']])

# Concatenate the scaled features back into a single array for model input
X_train_scaled = np.hstack((X_train_coords_scaled, X_train_steering_scaled))
X_test_scaled = np.hstack((X_test_coords_scaled, X_test_steering_scaled))

# Restore -999 in targets after scaling
y_coords_train.fillna(-999, inplace=True)
y_coords_test.fillna(-999, inplace=True)


# Build model
inputs = Input(shape=(X_train_scaled.shape[1],))
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
coords_output = Dense(4, name='coords_output')(x)

model = Model(inputs=inputs, outputs=coords_output)
model.compile(optimizer=Adam(), loss={'coords_output': custom_mse}, metrics={'coords_output': ['mean_absolute_error']})

# Train model
model.fit(X_train_scaled, y_coords_train, validation_split=0.2, epochs=20, batch_size=32)

# Evaluate model
results = model.evaluate(X_test_scaled, y_coords_test)
predictions_coords = model.predict(X_test_scaled)

# Calculate metrics
y_pred = model.predict(X_test_scaled)
mask = np.all(y_coords_test != -999, axis=1)
filtered_y_test = y_coords_test[mask]
filtered_y_pred = y_pred[mask]

filtered_mse = mean_squared_error(filtered_y_test, filtered_y_pred)
filtered_rmse = np.sqrt(filtered_mse)
filtered_mae = mean_absolute_error(filtered_y_test, filtered_y_pred)
filtered_r2 = r2_score(filtered_y_test, filtered_y_pred)

print(f"Filtered Mean Squared Error: {filtered_mse}")
print(f"Filtered Root Mean Squared Error: {filtered_rmse}")
print(f"Filtered Mean Absolute Error: {filtered_mae}")
print(f"Filtered R-squared Score: {filtered_r2}")


Epoch 1/20
414/414 [==============================] - 1s 775us/step - loss: 4511.8594 - mean_absolute_error: 100.7553 - val_loss: 566.7874 - val_mean_absolute_error: 73.4547
Epoch 2/20
414/414 [==============================] - 0s 710us/step - loss: 482.1287 - mean_absolute_error: 73.2666 - val_loss: 461.4359 - val_mean_absolute_error: 71.2615
Epoch 3/20
414/414 [==============================] - 0s 596us/step - loss: 408.7134 - mean_absolute_error: 71.7348 - val_loss: 351.9513 - val_mean_absolute_error: 69.0695
Epoch 4/20
414/414 [==============================] - 0s 634us/step - loss: 261.7370 - mean_absolute_error: 68.2698 - val_loss: 188.1418 - val_mean_absolute_error: 64.8292
Epoch 5/20
414/414 [==============================] - 0s 626us/step - loss: 154.8386 - mean_absolute_error: 64.6701 - val_loss: 135.5050 - val_mean_absolute_error: 62.4306
Epoch 6/20
414/414 [==============================] - 0s 606us/step - loss: 130.0042 - mean_absolute_error: 63.4324 - val_loss: 123.3358 -

In [26]:
def custom_sequential_predictions(model, scaler_coords, scaler_steering, X, start_index, n_predictions):
    predictions = []
    # Retrieve initial features and control inputs
    current_features = X.iloc[start_index:start_index + 1][['x_closest', 'x_middle1', 'x_middle2', 'x_farthest', 'steering_angle']]

    for i in range(n_predictions):
        # Separate coordinates and steering for scaling
        input_coords = current_features[['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']]
        input_steering = current_features[['steering_angle']]

        # Scale the features for model input
        scaled_coords = scaler_coords.transform(input_coords)
        scaled_steering = scaler_steering.transform(input_steering)

        # Combine scaled features for model input
        model_input = np.hstack((scaled_coords, scaled_steering))

        # Predict the next coordinates using the scaled features
        predicted_coords = model.predict(model_input)[0]
        predictions.append(predicted_coords)

        # Update current_features for the next prediction
        # Creating a new DataFrame for next prediction input
        next_features = pd.DataFrame([predicted_coords], columns=['x_closest', 'x_middle1', 'x_middle2', 'x_farthest'])
        next_features['steering_angle'] = X.iloc[start_index + i + 1]['steering_angle'] if start_index + i + 1 < len(X) else X.iloc[-1]['steering_angle']

        # Replace current features with next_features
        current_features = next_features

        print(f"Step {i + 1}: Predicted Coordinates: {predicted_coords}")

    return predictions


custom_df = pd.DataFrame({
    'x_closest': [120] * 10,  
    'x_middle1': [120] * 10,
    'x_middle2': [120] * 10,
    'x_farthest': [120] * 10,
    'steering_angle': [1] * 10 
})

start_index = 0
n_predictions = 10
custom_predictions = custom_sequential_predictions(model, scaler_coords, scaler_steering, custom_df, start_index, n_predictions)


1/1 [==============================] - 0s 13ms/step
Step 1: Predicted Coordinates: [122.283585 124.09933  123.67944  121.23071 ]
1/1 [==============================] - 0s 10ms/step
Step 2: Predicted Coordinates: [124.677734 127.570724 127.77012  124.96136 ]
1/1 [==============================] - 0s 12ms/step
Step 3: Predicted Coordinates: [127.01237 130.85974 131.9965  129.48508]
1/1 [==============================] - 0s 12ms/step
Step 4: Predicted Coordinates: [129.24684 134.05699 136.28326 134.29625]
1/1 [==============================] - 0s 12ms/step
Step 5: Predicted Coordinates: [131.36604 137.17892 140.5947  139.23627]
1/1 [==============================] - 0s 14ms/step
Step 6: Predicted Coordinates: [133.36241 140.2218  144.90662 144.24321]
1/1 [==============================] - 0s 13ms/step
Step 7: Predicted Coordinates: [135.23096 143.17719 149.19856 149.28232]
1/1 [==============================] - 0s 14ms/step
Step 8: Predicted Coordinates: [136.968   146.03569 153.4518  154

In [12]:
print("Feature Scaler Parameters:")
print("Minima:", scaler_coords.min_)
print("Scale factors:", scaler_coords.scale_)

print("\nSteering Scaler Parameters:")
print("Minima:", scaler_steering.min_)
print("Scale factors:", scaler_steering.scale_)


Feature Scaler Parameters:
Minima: [-0.01376147 -0.01376147 -0.01376147 -0.01382488]
Scale factors: [0.00458716 0.00458716 0.00458716 0.00460829]

Steering Scaler Parameters:
Minima: [0.5]
Scale factors: [0.5]


In [14]:
print("Shape of training data:", X_train_scaled.shape)
print("Shape of input features right before prediction:", scaled_features.shape)


Shape of training data: (16550, 5)
Shape of input features right before prediction: (1, 4)


In [15]:
print("Input features to the model:", scaled_features)


Input features to the model: [[0.54587156 0.54587156 0.54587156 0.5483871 ]]


In [16]:
test_input = np.array([[0.5, 0.5, 0.5, 0.5, 0.5]])  # Example simple input
try:
    test_output = model.predict(test_input)
    print("Output for test input:", test_output)
except Exception as e:
    print("Error during model prediction with test input:", e)


1/1 [==============================] - 0s 17ms/step
Output for test input: [[111.043785 112.21169  111.34141  112.31102 ]]


In [12]:
from tensorflow.keras.models import load_model


model.save('NN_MinMaxScale.h5')
print("Model saved successfully.")


Model saved successfully.


In [13]:
import pickle

# Save the scalers to disk
with open('scaler_coords_minmax.pkl', 'wb') as file:
    pickle.dump(scaler_coords, file)

with open('scaler_steering_minmax.pkl', 'wb') as file:
    pickle.dump(scaler_steering, file)


## NN with normalization and without throttle( Standard Scalar)


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

def custom_mse(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, -999), tf.float32)
    error = tf.square(y_pred - y_true) * mask
    return tf.reduce_sum(error) / tf.reduce_sum(mask)

# Load and preprocess data
lines = []
with open('Final_static_dataset.txt', 'r') as file:
    lines = file.readlines()

steering_angles, throttles, x_coords_list, flags_list = [], [], [], []
for line in lines:
    if line.strip():
        parts = line.split(': ')
        image_number, data = parts[0], parts[1]
        control_values, points, flags = eval(data)
        steering_angles.append(control_values[0])
        throttles.append(control_values[1])
        x_coords_list.append([point[0] for point in points])

df = pd.DataFrame(x_coords_list, columns=['x_closest', 'x_middle1', 'x_middle2', 'x_farthest'])
df['steering_angle'] = steering_angles

for col in ['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']:
    df[f'next_{col}'] = df[col].shift(-1)
df = df[:-1]

# Replace -999 with NaN for scaling purposes
df.replace(-999, np.nan, inplace=True)

y_coords = df[[f'next_{col}' for col in ['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']].copy()]
X = df[['x_closest', 'x_middle1', 'x_middle2', 'x_farthest', 'steering_angle']].copy()

# Split data
X_train, X_test, y_coords_train, y_coords_test = train_test_split(X, y_coords, test_size=0.2, random_state=42)

# Initialize two scalers
#scaler =  StandardScaler()

scaler_coords = StandardScaler()
scaler_steering = StandardScaler()

# Fit the scalers on the appropriate data
scaler_coords.fit(X_train[['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']].fillna(X_train.mean()))
scaler_steering.fit(X_train[['steering_angle']])

# Transform the data
X_train_coords_scaled = scaler_coords.transform(X_train[['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']].fillna(X_train.mean()))
X_train_steering_scaled = scaler_steering.transform(X_train[['steering_angle']])

X_test_coords_scaled = scaler_coords.transform(X_test[['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']].fillna(X_train.mean()))
X_test_steering_scaled = scaler_steering.transform(X_test[['steering_angle']])

# Concatenate the scaled features back into a single array for model input
X_train_scaled = np.hstack((X_train_coords_scaled, X_train_steering_scaled))
X_test_scaled = np.hstack((X_test_coords_scaled, X_test_steering_scaled))

# Restore -999 in targets after scaling
y_coords_train.fillna(-999, inplace=True)
y_coords_test.fillna(-999, inplace=True)


# Build model
inputs = Input(shape=(X_train_scaled.shape[1],))
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
coords_output = Dense(4, name='coords_output')(x)

model = Model(inputs=inputs, outputs=coords_output)
model.compile(optimizer=Adam(), loss={'coords_output': custom_mse}, metrics={'coords_output': ['mean_absolute_error']})

# Train model
model.fit(X_train_scaled, y_coords_train, validation_split=0.2, epochs=20, batch_size=32)

# Evaluate model
results = model.evaluate(X_test_scaled, y_coords_test)
predictions_coords = model.predict(X_test_scaled)

# Calculate metrics
y_pred = model.predict(X_test_scaled)
mask = np.all(y_coords_test != -999, axis=1)
filtered_y_test = y_coords_test[mask]
filtered_y_pred = y_pred[mask]

filtered_mse = mean_squared_error(filtered_y_test, filtered_y_pred)
filtered_rmse = np.sqrt(filtered_mse)
filtered_mae = mean_absolute_error(filtered_y_test, filtered_y_pred)
filtered_r2 = r2_score(filtered_y_test, filtered_y_pred)

print(f"Filtered Mean Squared Error: {filtered_mse}")
print(f"Filtered Root Mean Squared Error: {filtered_rmse}")
print(f"Filtered Mean Absolute Error: {filtered_mae}")
print(f"Filtered R-squared Score: {filtered_r2}")


Epoch 1/20


2024-07-22 17:36:57.494417: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


414/414 [==============================] - 1s 790us/step - loss: 4729.6240 - mean_absolute_error: 104.6386 - val_loss: 586.5490 - val_mean_absolute_error: 74.9460
Epoch 2/20
414/414 [==============================] - 0s 604us/step - loss: 307.4893 - mean_absolute_error: 69.9591 - val_loss: 182.5223 - val_mean_absolute_error: 64.8321
Epoch 3/20
414/414 [==============================] - 0s 575us/step - loss: 142.6162 - mean_absolute_error: 64.6290 - val_loss: 115.4601 - val_mean_absolute_error: 61.9966
Epoch 4/20
414/414 [==============================] - 0s 558us/step - loss: 107.3141 - mean_absolute_error: 62.9906 - val_loss: 98.0410 - val_mean_absolute_error: 61.1209
Epoch 5/20
414/414 [==============================] - 0s 560us/step - loss: 98.2452 - mean_absolute_error: 62.4460 - val_loss: 93.1610 - val_mean_absolute_error: 60.8490
Epoch 6/20
414/414 [==============================] - 0s 567us/step - loss: 94.3272 - mean_absolute_error: 62.2286 - val_loss: 89.8889 - val_mean_absolu

In [7]:
def custom_sequential_predictions(model, scaler_coords, scaler_steering, X, start_index, n_predictions):
    predictions = []
    # Retrieve initial features and control inputs
    current_features = X.iloc[start_index:start_index + 1][['x_closest', 'x_middle1', 'x_middle2', 'x_farthest', 'steering_angle']]

    for i in range(n_predictions):
        # Separate coordinates and steering for scaling
        input_coords = current_features[['x_closest', 'x_middle1', 'x_middle2', 'x_farthest']]
        input_steering = current_features[['steering_angle']]

        # Scale the features for model input
        scaled_coords = scaler_coords.transform(input_coords)
        scaled_steering = scaler_steering.transform(input_steering)

        # Combine scaled features for model input
        model_input = np.hstack((scaled_coords, scaled_steering))

        # Predict the next coordinates using the scaled features
        predicted_coords = model.predict(model_input)[0]
        predictions.append(predicted_coords)

        # Update current_features for the next prediction
        # Creating a new DataFrame for next prediction input
        next_features = pd.DataFrame([predicted_coords], columns=['x_closest', 'x_middle1', 'x_middle2', 'x_farthest'])
        next_features['steering_angle'] = X.iloc[start_index + i + 1]['steering_angle'] if start_index + i + 1 < len(X) else X.iloc[-1]['steering_angle']

        # Replace current features with next_features
        current_features = next_features

        print(f"Step {i + 1}: Predicted Coordinates: {predicted_coords}")

    return predictions


custom_df = pd.DataFrame({
    'x_closest': [120] * 10,  
    'x_middle1': [120] * 10,
    'x_middle2': [120] * 10,
    'x_farthest': [120] * 10,
    'steering_angle': [0] * 10 
})

start_index = 0
n_predictions = 10
custom_predictions = custom_sequential_predictions(model, scaler_coords, scaler_steering, custom_df, start_index, n_predictions)


1/1 [==============================] - 0s 14ms/step
Step 1: Predicted Coordinates: [118.809074 119.65357  117.84341  118.66243 ]
1/1 [==============================] - 0s 11ms/step
Step 2: Predicted Coordinates: [117.691574 119.031166 116.127716 116.22561 ]
1/1 [==============================] - 0s 9ms/step
Step 3: Predicted Coordinates: [116.62981  118.28177  114.651115 113.94006 ]
1/1 [==============================] - 0s 10ms/step
Step 4: Predicted Coordinates: [115.611244 117.45203  113.30537  111.949066]
1/1 [==============================] - 0s 9ms/step
Step 5: Predicted Coordinates: [114.62954 116.56326 112.02467 110.18133]
1/1 [==============================] - 0s 10ms/step
Step 6: Predicted Coordinates: [113.68325 115.63026 110.76768 108.54841]
1/1 [==============================] - 0s 10ms/step
Step 7: Predicted Coordinates: [112.77443  114.665886 109.5092   106.982   ]
1/1 [==============================] - 0s 10ms/step
Step 8: Predicted Coordinates: [111.90767 113.68244 108

In [8]:
from tensorflow.keras.models import load_model


model.save('NN_standscale.h5')
print("Model saved successfully.")


Model saved successfully.


In [9]:
import pickle

# Save the scalers to disk
with open('scaler_coords_standard.pkl', 'wb') as file:
    pickle.dump(scaler_coords, file)

with open('scaler_steering_standard.pkl', 'wb') as file:
    pickle.dump(scaler_steering, file)
